In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from textwrap import wrap
from matplotlib.lines import Line2D
from matplotlib.offsetbox import OffsetImage, AnnotationBbox


In [2]:
#21년 08월 데이터 로드
df_08 = pd.read_csv('seoul_bike_21_08.csv', encoding = 'cp949', low_memory = False)
#21년 09월 데이터 로드
df_09 = pd.read_csv('seoul_bike_21_09.csv', encoding = 'cp949', low_memory = False)
#08, 09월 데이터 열 병합
df_new = pd.concat([df_08,df_09])

In [3]:
#df_new.to_csv("df_new.csv", mode='w',encoding='cp949')

In [4]:
#info로 각 columns 별 dtype 확인.
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6865402 entries, 0 to 3645806
Data columns (total 11 columns):
 #   Column    Dtype  
---  ------    -----  
 0   자전거번호     object 
 1   대여일시      object 
 2   대여 대여소번호  int64  
 3   대여 대여소명   object 
 4   대여거치대     int64  
 5   반납일시      object 
 6   반납대여소번호   int64  
 7   반납대여소명    object 
 8   반납거치대     int64  
 9   이용시간      int64  
 10  이용거리      float64
dtypes: float64(1), int64(5), object(5)
memory usage: 628.5+ MB


In [5]:
#적재된 메모리 해제를 위한 delete
del df_08
del df_09

In [6]:
#필요한 컬럼만 추출해서 df_new로 대입.
df_new = df_new[['자전거번호', '대여일시','이용시간','이용거리']]

In [7]:
# check dtype
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6865402 entries, 0 to 3645806
Data columns (total 4 columns):
 #   Column  Dtype  
---  ------  -----  
 0   자전거번호   object 
 1   대여일시    object 
 2   이용시간    int64  
 3   이용거리    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 261.9+ MB


In [8]:
df_new.shape

(6865402, 4)

In [9]:
#고장 데이터셋 로드
df_break = pd.read_csv('seoul_bike_break.csv', encoding = 'utf-8')

In [10]:
df_break

,자전거번호,등록일시,고장구분
0,SPB-00108,2015-12-04 7:41,체인
1,SPB-00210,2015-12-07 12:23,체인
2,SPB-00035,2015-12-09 17:07,기타
3,SPB-01024,2015-12-13 21:02,기타
4,SPB-01015,2015-12-28 10:12,기타
...,...,...,...
181587,SPB-32255,2020-10-30 23:26,안장
181588,SPB-32255,2020-10-30 23:26,페달
181589,SPB-37980,2020-10-30 23:47,안장
181590,SPB-21980,2020-10-30 23:54,체인


__자전거번호, 등록일시 두 개의 변수를 이용하여 계산__

In [11]:
#df_break = df_break.sort_values(by=['등록일시'])
cols = ['자전거번호','등록일시']


In [12]:
df_break = df_break.sort_values(by=['자전거번호'])

tups = df_break[cols].sort_values(by=cols, ascending=(True, True)).apply(tuple, axis=1)
f, i = pd.factorize(tups)
display(tups, f, i)

10160     (SPB-00001, 2018-05-17 19:04)
10856     (SPB-00001, 2018-05-28 19:13)
10855     (SPB-00001, 2018-05-28 19:13)
12372     (SPB-00001, 2018-06-18 19:49)
12373     (SPB-00001, 2018-06-18 19:49)
                      ...              
181510    (SPB-51167, 2020-10-30 14:22)
181328    (SPB-51182, 2020-10-29 12:13)
181380    (SPB-51425, 2020-10-29 17:44)
181290     (SPB-51506, 2020-10-29 8:43)
181558    (SPB-51506, 2020-10-30 19:29)
Length: 181592, dtype: object

array([     0,      1,      1, ..., 163918, 163919, 163920], dtype=int64)

Index([('SPB-00001', '2018-05-17 19:04'), ('SPB-00001', '2018-05-28 19:13'),
       ('SPB-00001', '2018-06-18 19:49'), ('SPB-00001', '2018-06-22 17:11'),
        ('SPB-00001', '2018-06-23 8:52'),  ('SPB-00001', '2018-06-28 0:45'),
       ('SPB-00001', '2018-06-28 12:15'), ('SPB-00001', '2018-07-04 19:24'),
       ('SPB-00001', '2018-07-05 20:19'),  ('SPB-00001', '2018-07-06 1:37'),
       ...
        ('SPB-50796', '2020-10-27 7:33'),  ('SPB-50806', '2020-10-25 9:27'),
       ('SPB-50873', '2020-10-25 17:24'),  ('SPB-50902', '2020-10-24 8:05'),
       ('SPB-51122', '2020-10-30 18:43'), ('SPB-51167', '2020-10-30 14:22'),
       ('SPB-51182', '2020-10-29 12:13'), ('SPB-51425', '2020-10-29 17:44'),
        ('SPB-51506', '2020-10-29 8:43'), ('SPB-51506', '2020-10-30 19:29')],
      dtype='object', length=163921)

In [13]:
factorized = pd.Series(f + 1, tups.index)
display(factorized)

10160          1
10856          2
10855          2
12372          3
12373          3
           ...  
181510    163917
181328    163918
181380    163919
181290    163920
181558    163921
Length: 181592, dtype: int64

In [14]:
df_break['rank'] = factorized
df_break.sort_values(by='rank', ascending=True, inplace = True)

display(df_break)

,자전거번호,등록일시,고장구분,rank
10160,SPB-00001,2018-05-17 19:04,기타,1
10856,SPB-00001,2018-05-28 19:13,단말기,2
10855,SPB-00001,2018-05-28 19:13,페달,2
12375,SPB-00001,2018-06-18 19:49,페달,3
12377,SPB-00001,2018-06-18 19:49,기타,3
...,...,...,...,...
181510,SPB-51167,2020-10-30 14:22,단말기,163917
181328,SPB-51182,2020-10-29 12:13,단말기,163918
181380,SPB-51425,2020-10-29 17:44,기타,163919
181290,SPB-51506,2020-10-29 8:43,기타,163920


__rank로 오름차순 정렬하면 성공__

In [15]:
df_break.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 181592 entries, 10160 to 181558
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   자전거번호   181592 non-null  object
 1   등록일시    181592 non-null  object
 2   고장구분    181592 non-null  object
 3   rank    181592 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 6.9+ MB


In [16]:
df_break['등록일시'] = pd.to_datetime(df_break['등록일시'], format="%Y/%m/%d")


In [17]:
df_break['등록일시'].isna().sum()

0

In [18]:
df_break.groupby(['자전거번호'])

In [19]:
df_break['자전거번호']

10160     SPB-00001
10856     SPB-00001
10855     SPB-00001
12375     SPB-00001
12377     SPB-00001
            ...    
181510    SPB-51167
181328    SPB-51182
181380    SPB-51425
181290    SPB-51506
181558    SPB-51506
Name: 자전거번호, Length: 181592, dtype: object

In [20]:
#row 를 shift 해주면서 위, 아래 계산



In [21]:

#df_break['등록일시'] = pd.to_datetime(df_break['등록일시'], format="%Y/%m/%d")


In [22]:
#df_break = df_break['자전거번호'].groupby(df_break['등록일시']).sum()


In [23]:
df_break.head(50)

,자전거번호,등록일시,고장구분,rank
10160,SPB-00001,2018-05-17 19:04:00,기타,1
10856,SPB-00001,2018-05-28 19:13:00,단말기,2
10855,SPB-00001,2018-05-28 19:13:00,페달,2
12375,SPB-00001,2018-06-18 19:49:00,페달,3
12377,SPB-00001,2018-06-18 19:49:00,기타,3
12373,SPB-00001,2018-06-18 19:49:00,체인,3
12376,SPB-00001,2018-06-18 19:49:00,단말기,3
12372,SPB-00001,2018-06-18 19:49:00,타이어,3
12374,SPB-00001,2018-06-18 19:49:00,안장,3
12971,SPB-00001,2018-06-22 17:11:00,타이어,4


In [56]:
df_break.transpose()

,10856,12375,12971,13755,14358,15031,15690,17763,18443,19600,...,178445,177572,181057,179811,181147,179071,180882,180560,181218,181327
자전거번호,00001,00001,00001,00001,00001,00001,00001,00001,00001,00001,...,44835,44845,50101,50131,50222,50381,50437,50472,50504,50733
등록일시,2018/05/28,2018/06/18,2018/06/22,2018/06/28,2018/07/04,2018/07/08,2018/07/14,2018/07/27,2018/08/03,2018/08/13,...,2020/10/15,2020/10/11,2020/10/28,2020/10/21,2020/10/28,2020/10/18,2020/10/27,2020/10/25,2020/10/28,2020/10/29
고장구분,1,7,5,5,1,5,7,5,1,1,...,5,1,7,1,7,1,7,7,1,1
rank,2,3,4,6,8,11,13,16,17,19,...,163850,163858,163870,163876,163884,163895,163900,163903,163904,163910
diff,11,21,3,4,6,2,1,6,6,3,...,9,4,11,1,9,1,1,1,2,6
총_고장,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [24]:
df_break['diff'] = (df_break.등록일시 - df_break.등록일시.shift())

In [25]:
#timedelta to string
df_break['diff'] = df_break['diff'].astype(str)


In [26]:
df_break['diff'] = df_break['diff'].str.split(' ').str[0]

In [27]:
df_break.head(50)
#df_break['diff'] = df_break['diff'].astype(int)

,자전거번호,등록일시,고장구분,rank,diff
10160,SPB-00001,2018-05-17 19:04:00,기타,1,NaT
10856,SPB-00001,2018-05-28 19:13:00,단말기,2,11
10855,SPB-00001,2018-05-28 19:13:00,페달,2,0
12375,SPB-00001,2018-06-18 19:49:00,페달,3,21
12377,SPB-00001,2018-06-18 19:49:00,기타,3,0
12373,SPB-00001,2018-06-18 19:49:00,체인,3,0
12376,SPB-00001,2018-06-18 19:49:00,단말기,3,0
12372,SPB-00001,2018-06-18 19:49:00,타이어,3,0
12374,SPB-00001,2018-06-18 19:49:00,안장,3,0
12971,SPB-00001,2018-06-22 17:11:00,타이어,4,3


In [28]:
df_break.tail(50)
df_break['총_고장'] = 1

In [29]:
#고장구분 column LabelEncoder 작업
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_break['고장구분'] = le.fit_transform(df_break['고장구분'])
df_break['고장구분']

10160     0
10856     1
10855     7
12375     7
12377     0
         ..
181510    1
181328    1
181380    0
181290    0
181558    0
Name: 고장구분, Length: 181592, dtype: int32

In [30]:
#df_break['고장구분'] = 1

In [31]:
df_break['자전거번호'] = df_break['자전거번호'].str.split('-').str[1]
df_new['자전거번호'] = df_new['자전거번호'].str.split('-').str[1]

In [32]:
#t:m:s 날리기 위한 split 작업 후 대입
df_break['등록일시'] = df_break['등록일시'].dt.strftime('%Y/%m/%d')
df_break['등록일시']

10160     2018/05/17
10856     2018/05/28
10855     2018/05/28
12375     2018/06/18
12377     2018/06/18
             ...    
181510    2020/10/30
181328    2020/10/29
181380    2020/10/29
181290    2020/10/29
181558    2020/10/30
Name: 등록일시, Length: 181592, dtype: object

In [33]:
df_break

,자전거번호,등록일시,고장구분,rank,diff,총_고장
10160,00001,2018/05/17,0,1,NaT,1
10856,00001,2018/05/28,1,2,11,1
10855,00001,2018/05/28,7,2,0,1
12375,00001,2018/06/18,7,3,21,1
12377,00001,2018/06/18,0,3,0,1
...,...,...,...,...,...,...
181510,51167,2020/10/30,1,163917,-1,1
181328,51182,2020/10/29,1,163918,-2,1
181380,51425,2020/10/29,0,163919,0,1
181290,51506,2020/10/29,0,163920,-1,1


In [34]:
# 그룹별 drop_duplicates 필요.
# df_break['등록일시'] = df_break['등록일시'].drop_duplicates()

In [35]:
#df_break.dropna()

In [36]:
df_break = df_break.dropna()

In [41]:
df_break

,자전거번호,등록일시,고장구분,rank,diff,총_고장
10856,00001,2018/05/28,1,2,11,1
12375,00001,2018/06/18,7,3,21,1
12971,00001,2018/06/22,5,4,3,1
13755,00001,2018/06/28,5,6,4,1
14358,00001,2018/07/04,1,8,6,1
...,...,...,...,...,...,...
179071,50381,2020/10/18,1,163895,1,1
180882,50437,2020/10/27,7,163900,1,1
180560,50472,2020/10/25,7,163903,1,1
181218,50504,2020/10/28,1,163904,2,1


In [39]:
#NaT 결측치를 -99로 대체. (타입 통일 후, 형 변환을 위해)
df_break['diff'] = df_break['diff'].replace(['NaT'], '-99')


In [40]:
#조건문을 위한 타입변경
df_break['diff'] = df_break['diff'].astype(int)

# diff가 0이상인 애들만 따로 저장
diff_positive_num = df_break['diff'] > 0

#조건에 충족되는 값을 df_break에 저장.
df_break = df_break[diff_positive_num & df_break['고장구분'] == 1]
df_break

,자전거번호,등록일시,고장구분,rank,diff,총_고장
10856,00001,2018/05/28,1,2,11,1
12375,00001,2018/06/18,7,3,21,1
12971,00001,2018/06/22,5,4,3,1
13755,00001,2018/06/28,5,6,4,1
14358,00001,2018/07/04,1,8,6,1
...,...,...,...,...,...,...
179071,50381,2020/10/18,1,163895,1,1
180882,50437,2020/10/27,7,163900,1,1
180560,50472,2020/10/25,7,163903,1,1
181218,50504,2020/10/28,1,163904,2,1


In [42]:

#df_break.info()


### 현재 요까지 22.3.10 10:33 

In [43]:
#자전거번호, 등록일시, dfii 기준그룹화 한 값을
#df_group = df_break.groupby(['자전거번호','등록일시','diff']).sum()
group_col =  ['자전거번호','등록일시']
df_group = df_break.groupby(group_col).sum()
#df_group = df_break.groupby(['자전거번호']).sum()



In [48]:
#df_group.groupby(['자전거번호']).get_group('00001')

df_group.head(50)
#df_group['고장구분']
#df_group.reset_index()


고장구분  rank  diff  총_고장
자전거번호 등록일시                              
00001 2018/05/28     1     2    11     1
      2018/06/18     7     3    21     1
      2018/06/22     5     4     3     1
      2018/06/28     5     6     4     1
      2018/07/04     1     8     6     1
      2018/07/08     5    11     2     1
      2018/07/14     7    13     1     1
      2018/07/27     5    16     6     1
      2018/08/03     1    17     6     1
      2018/08/13     1    19     3     1
      2018/08/15     1    22     1     1
      2018/08/24     1    23     9     1
      2018/08/30     1    24     5     1
      2018/09/04     1    25     5     1
      2018/09/10     1    29     1     1
      2018/09/11     1    30     1     1
      2018/09/12     5    31     1     1
      2018/09/28     5    32    15     1
      2018/10/09     1    35     5     1
      2018/11/04     5    38     6     1
      2018/11/06     1    39     1     1
      2018/11/12     1    40     5     1
      2018/11/13     1    41     1     1
      2019/01/10     1    44     5     1
      2019/01/16     1    45     5     1
      2019/02/01     1    47     9     1
      2019/02/13     5    48    12     1
      2019/02/20     1    49     7     1
      2019/02/27     1    50     6     1
      2019/03/03     1    52     2     1
      2019/03/05     1    53     1     1
      2019/03/21     1    56     3     1
      2019/04/05     1    59     1     1
      2019/04/08     1    60     3     1
      2019/04/16     5    62     1     1
      2019/04/17     1    64     1     1
      2019/04/25     1    68     6     1
      2019/04/30     1    70     4     1
      2019/05/06     1    74     1     1
      2019/05/15     1    81     2     1
      2019/05/21     5    83     6     1
      2019/05/25     1    84     3     1
      2019/06/04     7    87     1     1
      2019/06/07     1    89     2     1
      2019/06/10     1    91     1     1
      2019/06/15     1    93     1     1
      2019/06/16     1    94     1     1
      2019/06/23     1    96     6     1
      2019/06/27     7   101     1     1
      2019/06/30     7   103     1     1

In [47]:
df_group['고장구분'].unique()

array([1, 7, 5, 3])

In [49]:
#df_group['diff2'] = (df_group.등록일시 - df_group.등록일시.shift())

AttributeError: 'DataFrame' object has no attribute '등록일시'

# 이까지만 보세용~~~~~~! 22.03.10 

In [52]:
df_group_reset = df_group.reset_index()

In [55]:
df_group_reset.tail(50)

,자전거번호,등록일시,고장구분,rank,diff,총_고장
65361,44469,2020/09/20,1,163640,10,1
65362,44481,2020/09/15,5,163646,4,1
65363,44502,2020/10/17,5,163660,61,1
65364,44509,2020/09/11,5,163664,12,1
65365,44513,2020/09/18,1,163667,1,1
65366,44513,2020/10/15,5,163669,22,1
65367,44532,2020/09/04,5,163684,6,1
65368,44536,2020/10/26,7,163691,25,1
65369,44538,2020/09/23,5,163694,9,1
65370,44538,2020/10/22,1,163695,29,1


In [ ]:
df_new['대여일시'] = pd.to_datetime(df_new['대여일시'], format="%Y/%m/%d")

In [ ]:
df_new.info()

In [ ]:
df_break_count = df_break.groupby(['자전거번호']).sum()

In [ ]:
df_break_count = df_break_count.reset_index()
#df_break에서 자전거번호별로 그룹화 후 날짜중복 제거 , 1로 묶을게요.

In [ ]:
df_break_count

In [ ]:
df_break_count

In [ ]:
#df_break 고장구분은 count로 처리하기 위해 1로 값 변경
#df_break['고장구분'] = 1

In [ ]:
#df_group = df_break.groupby(['자전거번호', '등록일시'])
#df_group = df_group.sum()
#df_group['고장구분'] = 1

In [ ]:
#df_group

In [ ]:
#df_new.rename(index = {'대여일시': '등록일시'}, inplace = True)


In [ ]:
df_merge = pd.merge(df_new,df_break, how = 'left', on = ("자전거번호"))

In [ ]:
df_merge

In [ ]:
df_merge2 = pd.merge(df_merge,df_break_count, how = 'left', on = ("자전거번호"))

In [ ]:
df_merge2

In [ ]:
df_merge2.info()

In [ ]:
df_merge.tail(50)

In [ ]:
df_merge = df_merge[['자전거번호', '등록일시','대여일시','이용시간','이용거리','고장구분']]

In [ ]:
df_merge.head()

In [ ]:
df_merge.sort_values(by=['자전거번호'], inplace = True, na_position='last', axis=0)


In [ ]:
df_merge.head(50)


In [ ]:
#df_merge.groupby(['자전거번호', '등록일시'])
# 그룹화 한번해보자
df_group = df_merge.groupby(['자전거번호', '등록일시']).sum()


In [ ]:
df_group

In [ ]:
print(df_merge.isnull())

In [ ]:
df_merge['고장구분'].isnull().sum()

In [ ]:
df_merge.info()

In [ ]:
df_merge.sample(50)